# 用 tf.data 加载 CSV 数据

这篇教程通过一个示例展示了怎样将 CSV 格式的数据加载进 `tf.data.Dataset`。

这篇教程使用的是泰坦尼克号乘客的数据。模型会根据乘客的年龄、性别、票务舱和是否独自旅行等特征来预测乘客生还的可能性。

## 设置

In [1]:
try:
  # Colab only
  %tensorflow_version 2.x
except Exception:
    pass


In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [3]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

16384/13049 [=====================================]  - 0s 0us/step


In [4]:
# 让 numpy 数据更易读。
np.set_printoptions(precision=3, suppress=True)

## 加载数据

开始的时候，我们通过打印 CSV 文件的前几行来了解文件的格式。

In [5]:
!head {train_file_path}

'head' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


正如你看到的那样，CSV 文件的每列都会有一个列名。dataset 的构造函数会自动识别这些列名。如果你使用的文件的第一行不包含列名，那么需要将列名通过字符串列表传给 `make_csv_dataset` 函数的 `column_names` 参数。

 



```python

CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

dataset = tf.data.experimental.make_csv_dataset(
     ...,
     column_names=CSV_COLUMNS,
     ...)
  
```


这个示例使用了所有的列。如果你需要忽略数据集中的某些列，创建一个包含你需要使用的列的列表，然后传给构造器的（可选）参数 `select_columns`。

```python

dataset = tf.data.experimental.make_csv_dataset(
  ...,
  select_columns = columns_to_use, 
  ...)

```

对于包含模型需要预测的值的列是你需要显式指定的。

In [6]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

现在从文件中读取 CSV 数据并且创建 dataset。

(完整的文档，参考 `tf.data.experimental.make_csv_dataset`)


In [7]:
def get_dataset(file_path):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=12, # 为了示例更容易展示，手动设置较小的值
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.


dataset 中的每个条目都是一个批次，用一个元组（*多个样本*，*多个标签*）表示。样本中的数据组织形式是以列为主的张量（而不是以行为主的张量），每条数据中包含的元素个数就是批次大小（这个示例中是 12）。

阅读下面的示例有助于你的理解。

In [8]:
examples, labels = next(iter(raw_train_data)) # 第一个批次
print("EXAMPLES: \n", examples, "\n")
print("LABELS: \n", labels)

EXAMPLES: 
 OrderedDict([('sex', <tf.Tensor: id=176, shape=(12,), dtype=string, numpy=
array([b'male', b'female', b'male', b'male', b'female', b'male', b'male',
       b'male', b'male', b'male', b'female', b'female'], dtype=object)>), ('age', <tf.Tensor: id=168, shape=(12,), dtype=float32, numpy=
array([40.5, 28. , 28. , 11. , 28. , 40. , 28. , 18. , 22. , 28. , 22. ,
       28. ], dtype=float32)>), ('n_siblings_spouses', <tf.Tensor: id=174, shape=(12,), dtype=int32, numpy=array([0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0])>), ('parch', <tf.Tensor: id=175, shape=(12,), dtype=int32, numpy=array([0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])>), ('fare', <tf.Tensor: id=173, shape=(12,), dtype=float32, numpy=
array([ 7.75 ,  7.75 ,  0.   , 18.788, 24.15 ,  0.   ,  7.55 ,  6.496,
        7.25 ,  8.05 , 55.   ,  7.787], dtype=float32)>), ('class', <tf.Tensor: id=170, shape=(12,), dtype=string, numpy=
array([b'Third', b'Third', b'Second', b'Third', b'Third', b'First',
       b'Third', b'Third', b'Third', b'Th

## 数据预处理

### 分类数据

CSV 数据中的有些列是分类的列。也就是说，这些列只能在有限的集合中取值。

使用 `tf.feature_column` API 创建一个 `tf.feature_column.indicator_column` 集合，每个 `tf.feature_column.indicator_column` 对应一个分类的列。


In [9]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}


In [10]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [11]:
# 你刚才创建的内容
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

这将是后续构建模型时处理输入数据的一部分。

### 连续数据

连续数据需要标准化。

写一个函数标准化这些值，然后将这些值改造成 2 维的张量。


In [12]:
def process_continuous_data(mean, data):
  # 标准化数据
  data = tf.cast(data, tf.float32) * 1/(2*mean)
  return tf.reshape(data, [-1, 1])

现在创建一个数值列的集合。`tf.feature_columns.numeric_column` API 会使用 `normalizer_fn` 参数。在传参的时候使用  [`functools.partial`](https://docs.python.org/3/library/functools.html#functools.partial)，`functools.partial` 由使用每个列的均值进行标准化的函数构成。

In [13]:
MEANS = {
    'age' : 29.631308,
    'n_siblings_spouses' : 0.545455,
    'parch' : 0.379585,
    'fare' : 34.385399
}

numerical_columns = []

for feature in MEANS.keys():
  num_col = tf.feature_column.numeric_column(feature, normalizer_fn=functools.partial(process_continuous_data, MEANS[feature]))
  numerical_columns.append(num_col)

In [14]:
# 你刚才创建的内容。
numerical_columns

[NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function process_continuous_data at 0x0000017C89B75730>, 29.631308)),
 NumericColumn(key='n_siblings_spouses', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function process_continuous_data at 0x0000017C89B75730>, 0.545455)),
 NumericColumn(key='parch', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function process_continuous_data at 0x0000017C89B75730>, 0.379585)),
 NumericColumn(key='fare', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function process_continuous_data at 0x0000017C89B75730>, 34.385399))]

这里使用标准化的方法需要提前知道每列的均值。如果需要计算连续的数据流的标准化的值可以使用 [TensorFlow Transform](https://www.tensorflow.org/tfx/transform/get_started)。

### 创建预处理层

将这两个特征列的集合相加，并且传给 `tf.keras.layers.DenseFeatures` 从而创建一个进行预处理的输入层。

In [15]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numerical_columns)

## 构建模型

从 `preprocessing_layer` 开始构建 `tf.keras.Sequential`。

In [16]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

## 训练、评估和预测

现在可以实例化和训练模型。

In [17]:
train_data = raw_train_data.shuffle(500)
test_data = raw_test_data

In [18]:
model.fit(train_data, epochs=20)

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Epoch 1/20
53/53 [==============================]: 0.7810 - accuracy: 0.333 - 1s 467ms/step - loss: 0.7992 - accuracy: 0.266 - 1s 313ms/step - loss: 0.7767 - accuracy: 0.296 - 1s 236ms/step - loss: 0.7476 - accuracy: 0.384 - 1s 190ms/step - loss: 0.7384 - accuracy: 0.431 - 1s 159ms/step - loss: 0.7281 - accuracy: 0.476 - 1s 137ms/step - loss: 0.7228 - accuracy: 0.480 - 1s 121ms/step - loss: 0.7111 - accuracy: 0.528 - 1s 108ms/step - loss: 0.7051 - accuracy: 0.555 - 1s 98ms/step - loss: 0.6945 - accuracy: 0.576 - 1s 90ms/step - loss: 0.6827 - accuracy: 0.61 - 1s 83ms/step - loss: 0.6845 - accuracy: 0.60 - 1s 77ms/step - loss: 0.6810 - accuracy: 0.60 - 1s 72ms/step - loss: 0.6751 - accuracy: 0.62 - 1s 67ms/step - loss: 0.6732 - accuracy: 0.62 -

当模型训练完成的时候，你可以在测试集 `test_data` 上检查准确性。

In [19]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

22/22 [==============================]: 0.6116 - accuracy: 0.583 - 0s 105ms/step - loss: 0.6170 - accuracy: 0.666 - 0s 71ms/step - loss: 0.4996 - accuracy: 0.750 - 0s 55ms/step - loss: 0.5170 - accuracy: 0.75 - 0s 44ms/step - loss: 0.4966 - accuracy: 0.75 - 0s 38ms/step - loss: 0.4941 - accuracy: 0.75 - 0s 33ms/step - loss: 0.4504 - accuracy: 0.77 - 0s 29ms/step - loss: 0.4474 - accuracy: 0.78 - 0s 27ms/step - loss: 0.4333 - accuracy: 0.79 - 0s 24ms/step - loss: 0.4192 - accuracy: 0.80 - 0s 22ms/step - loss: 0.3951 - accuracy: 0.82 - 0s 21ms/step - loss: 0.3905 - accuracy: 0.83 - 0s 20ms/step - loss: 0.4224 - accuracy: 0.82 - 0s 18ms/step - loss: 0.4198 - accuracy: 0.82 - 0s 17ms/step - loss: 0.4204 - accuracy: 0.82 - 0s 17ms/step - loss: 0.4077 - accuracy: 0.83 - 0s 16ms/step - loss: 0.4391 - accuracy: 0.82 - 0s 15ms/step - loss: 0.4460 - accuracy: 0.81 - 0s 15ms/step - loss: 0.4355 - accuracy: 0.82 - 0s 14ms/step - loss: 0.4378 - accuracy: 0.81 - 0s 14ms/step - loss: 0.4293 - accurac

使用 `tf.keras.Model.predict` 推断一个批次或多个批次的标签。

In [20]:
predictions = model.predict(test_data)

# 显示部分结果
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))



Predicted survival: 99.60%  | Actual outcome:  DIED
Predicted survival: 43.89%  | Actual outcome:  DIED
Predicted survival: 44.95%  | Actual outcome:  SURVIVED
Predicted survival: 2.34%  | Actual outcome:  DIED
Predicted survival: 5.64%  | Actual outcome:  DIED
Predicted survival: 38.27%  | Actual outcome:  DIED
Predicted survival: 99.85%  | Actual outcome:  SURVIVED
Predicted survival: 7.53%  | Actual outcome:  DIED
Predicted survival: 99.58%  | Actual outcome:  DIED
Predicted survival: 42.32%  | Actual outcome:  DIED
